# Refer /Keras /1. preProcess with keras

In [1]:
import os
from tqdm import tqdm
import pandas as pd 
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, GRU,LSTM, Embedding,Flatten, Dropout
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
%matplotlib inline

import pickle
import h5py

from tensorflow.keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir='logs/keras-lstm-3.2')

Using TensorFlow backend.


# make results reproducable

In [2]:

from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [3]:
INPUT_TEXT_LENGTH = 695 # from previous notebook
# vocab size needs to be taken care off

In [4]:
df_preprocessed = pd.read_pickle("./pickles/1_processed_pos_tag.pkl")

In [5]:
df_preprocessed.head()

,review,word_count,sentiment
0,plot two teen couple go church party drink dri...,358,0.0
1,happy bastard 's quick movie review damn y2k b...,144,0.0
2,movie like make jaded movie viewer thankful in...,276,0.0
3,`` quest camelot `` warner bros first feature-...,313,0.0
4,synopsis mentally unstable man undergo psychot...,402,0.0


In [6]:
VOCAB_SIZE = 10000

# Vectorising usin Keras
- Setting integer value to a string token

In [7]:
%%time 
# create the tokenizer
tokenizer =Tokenizer(VOCAB_SIZE)
# fit the tokenizer on the documents
tokenizer.fit_on_texts(df_preprocessed["review"])

Wall time: 1.24 s


In [8]:
#oredered as per max. freq
len(tokenizer.word_index)

34094

# Deciding Vocabulary for EMB. matrix

In [9]:
#len(tokenizer.word_index)
tokenizer.word_index

{"'s": 1,
 'film': 2,
 'movie': 3,
 "n't": 4,
 'one': 5,
 'make': 6,
 'like': 7,
 'character': 8,
 'get': 9,
 'see': 10,
 'go': 11,
 'time': 12,
 'good': 13,
 'even': 14,
 'scene': 15,
 'well': 16,
 'story': 17,
 'would': 18,
 'play': 19,
 'take': 20,
 'much': 21,
 'also': 22,
 'come': 23,
 'give': 24,
 'two': 25,
 'way': 26,
 'know': 27,
 'first': 28,
 'bad': 29,
 'seem': 30,
 'look': 31,
 'end': 32,
 'life': 33,
 'year': 34,
 'work': 35,
 'thing': 36,
 'could': 37,
 'plot': 38,
 'say': 39,
 'find': 40,
 'really': 41,
 'little': 42,
 'show': 43,
 'people': 44,
 'man': 45,
 'think': 46,
 'never': 47,
 'star': 48,
 'love': 49,
 'director': 50,
 'great': 51,
 'best': 52,
 'new': 53,
 'try': 54,
 'performance': 55,
 'big': 56,
 'many': 57,
 'action': 58,
 'actor': 59,
 'u': 60,
 'want': 61,
 'watch': 62,
 'role': 63,
 'another': 64,
 'use': 65,
 'back': 66,
 'become': 67,
 'audience': 68,
 'world': 69,
 'something': 70,
 'still': 71,
 'act': 72,
 'day': 73,
 'turn': 74,
 "'re": 75,
 'howe

# Converting text data to abv to integer values

In [14]:
%%time
seq = tokenizer.texts_to_sequences(df_preprocessed["review"])

Wall time: 859 ms


# Check for values

In [11]:
 df_preprocessed['review'][1]

"happy bastard 's quick movie review damn y2k bug 's got head start movie star jamie lee curtis another baldwin brother william time story regard crew tugboat come across deserted russian tech ship strangeness kick power back little know power within go gore bring action sequence virus still feel empty like movie go flash substance n't know crew really middle nowhere n't know origin take ship big pink flashy thing hit mir course n't know donald sutherland stumble around drunkenly throughout 's `` hey let 's chase people around robot `` acting average even like curtis 're likely get kick work halloween h20 sutherland waste baldwin well 's act like baldwin course real star stan winston 's robot design schnazzy cgi occasional good gore shot like pick someone 's brain robot body part really turn 's movie otherwise 's pretty much sunken ship movie"

In [16]:
len(max(df_preprocessed['review']))

2278

In [12]:
np.array(seq[1])

array([ 544, 2388,    1,  890,    3,  264, 1095,  943,    1, 4785,  181,
        119,    3,   48, 2666,  442, 2961,   64, 1400,  234,  588,   12,
         17, 1847,  555,   23,  538, 1541, 3416,  395, 1022,  320,   66,
         42,   27,  320,  524,   11, 1153,  178,   58,  139, 1560,   71,
        104, 1783,    7,    3,   11, 1414, 1615,    4,   27,  555,   41,
        531, 1289,    4,   27, 6713,   20,  395,   56, 4198, 2667,   36,
        280, 9313,  141,    4,   27, 2886, 2814, 1542,   88,  389,    1,
       1742,  187,    1,  441,   44,   88, 1802, 1561, 1063,   14,    7,
       2961,   75,  743,    9, 1022,   35, 2094, 8609, 2814,  434, 1400,
         16,    1,   72,    7, 1400,  141,   83,   48, 6714,    1, 1802,
        812, 2294, 1923,   13, 1153,  214,    7,  593,  279,    1, 1023,
       1802,  368,   81,   41,   74,    1,    3,  795,    1,  192,   21,
        395,    3])

In [13]:
'''
WE CAN SEE THE DIFFERENCE 10 , BCOZ OTHER VALUES ARE GRATER THEN 10000 SO ITS NOT CONSIDERED
'''
print(df_preprocessed['word_count'][1])
print(len(np.array(seq[1])))

144
134


In [14]:
print(" Actual VOCAB_SIZE deined: ",VOCAB_SIZE)
print(" Actual tokens created: ",len(tokenizer.word_index))# IF WE WANT TO CONSIDER EVERY TEXT IN VOCAB PASS THIS

 Actual VOCAB_SIZE deined:  10000
 Actual tokens created:  34094


# Padding and Truncating Data
* To feed it to N.N, inputs to have the same length
 - Either we ensure that all sequences in the entire data-set have the same length
 - Or Entier batch should be of same length
* Going about choosing ampunt to pad
 - going with longest seq, would be just waste of memory for texr whose length is small
 - going with smalles seq , would be just ignoring other imp values 
 - so we go optimal


In [15]:
'''
All that the Embedding layer does is to map the integer inputs to the vectors found at the corresponding index in the 
embedding matrix, i.e. the sequence [1, 2] would be converted to [embeddings[1], embeddings[2]]
'''
print(np.array(seq[506]))

[   2 5595 6919  146   11  434  240]


# Padding and trucating here

In [16]:
imdb_data_pad = sequence.pad_sequences(seq , maxlen=INPUT_TEXT_LENGTH,padding='pre', truncating='pre')

In [17]:
imdb_data_pad.shape
# 2k review
# and 695 fixed I/P shape

(2000, 695)

In [18]:
#checking
#imdb_data_pad[4]
type(seq)

list

In [19]:
tokenizer

# Tokenizer Inverse Map

In [20]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [21]:

def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

In [22]:
tokens_to_string(seq[506])

"film extraordinarily horrendous 'm go waste word"

In [23]:
# N.N Model
len(tokenizer.word_index) # This are total word in dict
#it depends if i want to use entier dict or only few occuring word

34094

# Split data to test/ train / dev

In [24]:
X= imdb_data_pad #PADDED VERSION OF DATA
y= df_preprocessed['sentiment'] # LABELS OF DATA

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

In [25]:
'''
def export_test_train_data_to_h5(filename, X_train, X_test, y_train, y_test):
    #filename has to be xyzw.h5 with extension
    h5f = h5py.File(filename, 'w')
    h5f.create_dataset('X', data=X)           #np.ndarry
    h5f.create_dataset('y', data=np.array(y)) #
    h5f.create_dataset('X_train', data=X_train)           #np.ndarry
    h5f.create_dataset('y_train', data=np.array(y_train)) #pd df
    h5f.create_dataset('X_test', data=X_test)
    h5f.create_dataset('y_test', data=np.array(y_test))
    # export weights of embedding_matrix incase of  
    h5f.close()
    print("file created")

filename = "2_2_keras.h5"
export_test_train_data_to_h5(filename,X_train, X_test, y_train, y_test)


'''

'\ndef export_test_train_data_to_h5(filename, X_train, X_test, y_train, y_test):\n    #filename has to be xyzw.h5 with extension\n    h5f = h5py.File(filename, \'w\')\n    h5f.create_dataset(\'X\', data=X)           #np.ndarry\n    h5f.create_dataset(\'y\', data=np.array(y)) #\n    h5f.create_dataset(\'X_train\', data=X_train)           #np.ndarry\n    h5f.create_dataset(\'y_train\', data=np.array(y_train)) #pd df\n    h5f.create_dataset(\'X_test\', data=X_test)\n    h5f.create_dataset(\'y_test\', data=np.array(y_test))\n    # export weights of embedding_matrix incase of  \n    h5f.close()\n    print("file created")\n\nfilename = "2_2_keras.h5"\nexport_test_train_data_to_h5(filename,X_train, X_test, y_train, y_test)\n\n\n'

# CREATING A MODEL
* http://fizzylogic.nl/2017/05/08/monitor-progress-of-your-keras-based-neural-network-using-tensorboard/

In [26]:
%%time
model = Sequential()
                   #vocab_size
model.add(Embedding(VOCAB_SIZE, 100, input_length=INPUT_TEXT_LENGTH))
#model.add(Flatten())

model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))


model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 695, 100)          1000000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 695, 100)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 1,080,501
Trainable params: 1,080,501
Non-trainable params: 0
_________________________________________________________________
None
Wall time: 360 ms


# print out model image

In [27]:
from keras.utils import plot_model
import os
#install graph viz locally 1st
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'# install 
plot_model(model, to_file='3_2_LSTM_model.png')

In [28]:
%%time

#tensorboard = TensorBoard(log_dir='logs/3_2_LSTM_model')
history = model.fit(X_train, y_train, epochs=3, batch_size=64, verbose=2,callbacks=[tensorboard]) 
# use validation_split=0.08 only incase data is not been splitte earlier


# TENSORBOARD
'''
# tensorboard --logdir logs/
# http://localhost:6006/
'''

Epoch 1/3
 - 21s - loss: 0.6924 - acc: 0.5164
Epoch 2/3
 - 20s - loss: 0.6301 - acc: 0.7664
Epoch 3/3
 - 20s - loss: 0.3341 - acc: 0.8903
Wall time: 1min 7s


In [29]:

# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))


Accuracy: 72.27%


In [ ]:
'''
Neural networks are stochastic, they can produce different results when the same model is fit on the same data.
This is mainly because of the random initial weights and the shuffling of patterns during mini-batch gradient descent. 
This means that any one scoring of a model is unreliable and we should estimate model skill based on an average of multiple runs.
'''

In [30]:
scores

[0.6407241041010077, 0.7227272734497533]

In [ ]:
#refs
#use tensorboard
# 1. https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/
# 2. Logloss is a very useful measure for evaluating the performance of learning algorithms on multi-class classification problems:
# 3. http://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html#sklearn.metrics.log_loss

# testing performance

In [57]:
len(X_test)

660

In [58]:
%%time
'''
#gives exact probab distribution
y_pred = model.predict(x=X_test[0:500])
y_pred = y_pred.T[0]
OR
y_pred_prob = model.predict_proba(x=X_test[0:500])
y_pred_prob = y_pred_prob.T[0]
'''
# outputs by N.N directly to which class it belongs to
y_pred_classes = model.predict_classes(x=X_test[0:660])
cls_pred = y_pred_classes.T[0]

Wall time: 5.54 s


In [59]:
# taking the actual true label
cls_true = np.array(y_test[0:660])

In [60]:
# comparing 2 classes
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]

In [61]:
len(incorrect)

136

In [62]:
incorrect  # all beow are idexes of mis-classified text

array([  1,   9,  11,  12,  16,  18,  21,  34,  36,  37,  38,  44,  50,
        56,  58,  60,  63,  64,  72,  73,  83,  86,  89,  94,  95,  98,
       103, 111, 113, 120, 122, 125, 128, 131, 133, 135, 136, 138, 139,
       147, 149, 153, 159, 164, 165, 167, 169, 172, 179, 183, 186, 189,
       192, 199, 201, 205, 206, 207, 208, 210, 215, 224, 230, 231, 233,
       241, 244, 247, 250, 255, 258, 261, 262, 263, 265, 274, 278, 283,
       291, 297, 298, 303, 305, 306, 312, 316, 317, 321, 326, 332, 338,
       339, 340, 347, 348, 349, 352, 353, 359, 360, 366, 368, 374, 376,
       377, 378, 380, 396, 408, 409, 412, 413, 419, 422, 426, 427, 429,
       431, 432, 435, 436, 444, 447, 453, 455, 458, 463, 481, 484, 485,
       488, 489, 490, 492, 496, 498], dtype=int64)

In [63]:
# direct checking from main df as X_test containt int representation of text
 df_preprocessed['review'][488]

"arrive barrage hype blair witch project one big box office success year however like golden child although blair witch make lot money 's good donahue williams leonard play three student set make documentary blair witch myth film make camcorder footage record mean grainy footage woozy camera angle although event start normal get weird pretty quickly threesome argue journey go although interest premise blair witch project amount nothing missed opportunity big mistake film make let three mediocre actor chance improvise dialogue ad-libbed still manage sound like poor b-movie 'script eventually degenerate shout match f-word include lot sound like student breaks argument creepy event occur 's back shout swear get tiresome quickly want see three people get lose wood shout lot swear 'd go scout camp audience mean get entertainment factor 'm quite sure supernatural part film actually interest especially close attention pay story develop first twenty minute music budget work film depend natural

In [64]:
print("actual sentiment ",df_preprocessed['sentiment'][488])
print("predicted sentiment ", cls_pred[488])
# negative sentiment are zero


actual sentiment  0.0
predicted sentiment  1


660